# Notebook 03: RAG con Prompt Chaining — Agente que Busca, Evalua y Corrige

---

## Objetivos de aprendizaje

Al finalizar este notebook seras capaz de:

1. **Entender por que RAG naive no es suficiente** y como prompt chaining agrega control de calidad en cada etapa del pipeline.
2. **Implementar un sistema Self-RAG** con autocorreccion: el agente evalua sus propios resultados y decide si necesita re-intentar.
3. **Usar LangGraph para orquestar la cadena** de pasos, incluyendo edges condicionales que crean loops de correccion.
4. **Diagnosticar y trazar** el flujo completo: saber exactamente por que el agente tomo cada decision.

### Prerequisitos

Este notebook asume que completaste:
- **Notebook 01**: Bases de datos vectoriales, embeddings, ChromaDB.
- **Notebook 02**: Pipeline RAG completo (indexacion, retrieval, generacion).

Aqui vamos un paso mas alla: no solo generamos respuestas, sino que las **evaluamos y corregimos** automaticamente.

### Referencia

- Asai, A. et al. (2023). *Self-RAG: Learning to Retrieve, Generate, and Critique*. arXiv:2310.11511.
- Chip Huyen (2024). *AI Engineering*, Cap. 10: RAG patterns in production.

---

## 1. Por que Prompt Chaining para RAG?

### El problema del RAG naive

En el Notebook 02 construimos un pipeline RAG funcional:

```
Query → Retrieval → Generacion → Respuesta
```

Este pipeline funciona, pero tiene **cero control de calidad**:

- Si el retriever trae documentos irrelevantes, el LLM genera basura.
- Si el LLM alucina a pesar de tener buen contexto, nadie lo detecta.
- Si el query es ambiguo, nadie lo refina antes de buscar.

En produccion, esto es inaceptable. Un chatbot de soporte que da informacion incorrecta sobre precios o politicas puede costarle dinero real a la empresa.

### La solucion: Prompt Chaining con validacion

Prompt chaining descompone el pipeline en pasos discretos, donde **cada paso valida y potencialmente corrige** al anterior:

```
┌──────────────┐     ┌───────────┐     ┌──────────────┐     ┌───────────┐     ┌──────────────┐     ┌──────────┐
│   Analisis   │────→│ Retrieval │────→│   Evaluar    │────→│  Generar  │────→│   Evaluar    │────→│  Output  │
│  del Query   │     │           │     │  Contexto    │     │ Respuesta │     │  Respuesta   │     │  Final   │
└──────────────┘     └───────────┘     └──────┬───────┘     └───────────┘     └──────┬───────┘     └──────────┘
       ^                                      │                                      │
       │                                      │                                      │
       └──── Contexto irrelevante? ───────────┘                                      │
                  (re-query)                                                         │
                                                          Alucinacion? ──────────────┘
                                                           (re-generate)
```

### Que hace cada paso

| Paso | Funcion | Que valida |
|------|---------|------------|
| **Query Analysis** | Refina el query del usuario para mejorar retrieval | Claridad y especificidad del query |
| **Retrieval** | Busca documentos relevantes en la base vectorial | - |
| **Grade Context** | Evalua si los documentos recuperados son relevantes | Relevancia del contexto vs. query |
| **Generate** | Genera respuesta basada SOLO en el contexto | - |
| **Grade Answer** | Evalua si la respuesta es fiel al contexto (no alucina) | Faithfulness de la respuesta |

### Cuando usar Prompt Chaining para RAG

**SI usalo cuando:**
- El costo de una respuesta incorrecta es alto (soporte al cliente, asesor legal, chatbot medico)
- Necesitas trazabilidad completa (por que el agente respondio X)
- La base de conocimiento es heterogenea y el retrieval puede fallar

**NO lo uses cuando:**
- La latencia debe ser sub-segundo (chaining agrega 3-8 segundos)
- El presupuesto de tokens es extremadamente limitado (cada evaluacion consume tokens)
- Las queries son simples y el retrieval es consistentemente bueno

---

## 2. Arquitectura del Agente

Vamos a implementar esto como un **state machine** con LangGraph. Cada nodo es una funcion que transforma el estado, y los edges condicionales deciden el flujo.

### State Machine completa

```
                    ┌───────────────────────────────────────────────────────────────┐
                    │                   LangGraph State Machine                     │
                    │                                                               │
                    │   ┌──────────────┐                                            │
                    │   │ analyze_query│◄───────────────────────────┐               │
                    │   └──────┬───────┘                           │               │
                    │          │                                    │               │
                    │          ▼                                    │               │
                    │   ┌──────────────┐                           │               │
                    │   │   retrieve   │                           │               │
                    │   └──────┬───────┘                           │               │
                    │          │                                    │               │
                    │          ▼                                    │               │
                    │   ┌──────────────┐    irrelevant    ┌────────┴──────┐        │
                    │   │ grade_context│──────────────────→│ should_regen  │        │
                    │   └──────┬───────┘                  │ (routing fn)  │        │
                    │          │ relevant                  └───────────────┘        │
                    │          ▼                                                    │
                    │   ┌──────────────┐                                            │
                    │   │   generate   │                                            │
                    │   └──────┬───────┘                                            │
                    │          │                                                    │
                    │          ▼                                                    │
                    │   ┌──────────────┐   hallucination  ┌────────────────┐        │
                    │   │ grade_answer │──────────────────→│ should_rewrite │        │
                    │   └──────┬───────┘                  │ (routing fn)   │        │
                    │          │ faithful                  └──────┬─────────┘        │
                    │          ▼                                  │ (re-generate)    │
                    │      ┌───────┐                             │                  │
                    │      │  END  │◄────────────────────────────┘ (finish)         │
                    │      └───────┘                                                │
                    └───────────────────────────────────────────────────────────────┘
```

### El estado que fluye entre nodos

Todos los nodos leen y escriben en un diccionario tipado (`RAGState`). Esto es clave: **cada nodo solo modifica su parte del estado** y el siguiente nodo puede leer todo lo que paso antes.

Campos del estado:
- `query`: el query original del usuario
- `refined_query`: version mejorada del query (post-analisis)
- `retrieved_docs`: documentos recuperados del vector store
- `context_grade`: "relevant" o "irrelevant"
- `generation`: la respuesta generada
- `answer_grade`: "faithful" o "hallucination"
- `sources`: fuentes citadas en la respuesta
- `retry_count` / `max_retries`: control de loops infinitos
- `steps_trace`: registro de cada paso ejecutado (para debugging)

### Edges condicionales

Los edges condicionales son **funciones de routing** que deciden el proximo nodo basandose en el estado:

1. **Despues de `grade_context`**: Si el contexto es irrelevante y no hemos agotado reintentos → volver a `analyze_query`. Si es relevante → ir a `generate`.
2. **Despues de `grade_answer`**: Si la respuesta alucina y no hemos agotado reintentos → volver a `generate`. Si es fiel → terminar.

---

## 3. Setup

In [ ]:
# =============================================================================
# Imports necesarios
# =============================================================================
import os
import json
from typing import Literal
from pathlib import Path

# LangGraph - orquestacion del grafo
from langgraph.graph import StateGraph, END

# LangChain - LLM y text processing
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Pydantic - structured output para grading
from pydantic import BaseModel, Field

# TypedDict para el estado del grafo
from typing import TypedDict

# ChromaDB - base de datos vectorial
import chromadb

# OpenAI - para embeddings y llamadas directas
from openai import OpenAI

# Utilidades
from IPython.display import display, Markdown, Image
from dotenv import load_dotenv

# =============================================================================
# Cargar variables de entorno
# =============================================================================
load_dotenv()

# Inicializar clientes
openai_client = OpenAI()  # Usa OPENAI_API_KEY del .env
llm = ChatOpenAI(
    model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
    temperature=0  # Determinismo para evaluaciones de calidad
)

# Verificar conexion
print("=" * 60)
print("SETUP COMPLETADO")
print("=" * 60)
print(f"Modelo LLM: {llm.model_name}")
print(f"Temperatura: {llm.temperature}")
print(f"API Key configurada: {'SI' if os.getenv('OPENAI_API_KEY') else 'NO'}")
print("=" * 60)

---

## 4. Preparar Base de Conocimiento

Vamos a indexar los mismos documentos que usamos en los notebooks anteriores: la base de conocimiento de productos y la guia tecnica de NovaTech. Usamos `RecursiveCharacterTextSplitter` para hacer chunking respetando la estructura del documento.

In [ ]:
# =============================================================================
# Cargar documentos desde ../data/
# =============================================================================
DATA_DIR = Path("../data")

# Leer ambos archivos .md
archivos_md = list(DATA_DIR.glob("*.md"))
print(f"Archivos encontrados: {len(archivos_md)}")

documentos_raw = []
metadatos_raw = []

for archivo in archivos_md:
    contenido = archivo.read_text(encoding="utf-8")
    documentos_raw.append(contenido)
    metadatos_raw.append({"source": archivo.name})
    print(f"  - {archivo.name}: {len(contenido):,} caracteres")

# =============================================================================
# Chunking con RecursiveCharacterTextSplitter
# =============================================================================
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # Tamano maximo por chunk
    chunk_overlap=50,     # Overlap para no perder contexto entre chunks
    separators=["\n## ", "\n### ", "\n\n", "\n", ". ", " ", ""],
)

# Procesar cada documento
all_chunks = []       # Textos de los chunks
all_metadatas = []    # Metadata por chunk (source + chunk_index)
all_ids = []          # IDs unicos

chunk_counter = 0
for doc_text, doc_meta in zip(documentos_raw, metadatos_raw):
    chunks = splitter.split_text(doc_text)
    for i, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        all_metadatas.append({
            "source": doc_meta["source"],
            "chunk_index": i,
        })
        all_ids.append(f"chunk_{chunk_counter}")
        chunk_counter += 1

print(f"\nTotal de chunks generados: {len(all_chunks)}")
print(f"Tamano promedio por chunk: {sum(len(c) for c in all_chunks) // len(all_chunks)} caracteres")

# =============================================================================
# Indexar en ChromaDB
# =============================================================================
chroma_client = chromadb.Client()  # Cliente en memoria (para desarrollo)

# Eliminar coleccion si existe (para re-ejecucion limpia)
try:
    chroma_client.delete_collection(name="novatech_kb")
except Exception:
    pass

collection = chroma_client.create_collection(
    name="novatech_kb",
    metadata={"hnsw:space": "cosine"}  # Similitud coseno
)

# Agregar todos los chunks a la coleccion
collection.add(
    documents=all_chunks,
    metadatas=all_metadatas,
    ids=all_ids,
)

print(f"\nColeccion '{collection.name}' creada con {collection.count()} documentos indexados.")
print("Base de conocimiento lista para retrieval.")

---

## 5. Definir Estado del Grafo

En LangGraph, el estado es un `TypedDict` que actua como la "memoria compartida" entre todos los nodos. Cada nodo recibe el estado completo, lo modifica parcialmente, y lo devuelve.

Piensa en el estado como una **ficha medica**: cada doctor (nodo) la lee, agrega sus notas, y la pasa al siguiente.

```
RAGState = {
    query:          "pregunta original del usuario"
    refined_query:  "pregunta mejorada por el analizador"
    retrieved_docs: ["doc1", "doc2", ...]
    context_grade:  "relevant" | "irrelevant"
    generation:     "respuesta generada por el LLM"
    answer_grade:   "faithful" | "hallucination"
    sources:        ["fuente1.md", "fuente2.md"]
    retry_count:    0, 1, 2, ...
    max_retries:    3
    steps_trace:    ["analyze_query", "retrieve", ...]
}
```

In [ ]:
# =============================================================================
# Definicion del estado del grafo (TypedDict)
# =============================================================================

class RAGState(TypedDict):
    """Estado compartido entre todos los nodos del grafo Self-RAG.
    
    Cada nodo lee el estado, lo modifica parcialmente, y lo retorna.
    LangGraph se encarga de hacer merge automatico.
    """
    query: str                    # Query original del usuario
    refined_query: str            # Query mejorada (post-analysis)
    retrieved_docs: list[str]     # Documentos recuperados del vector store
    context_grade: str            # "relevant" | "irrelevant"
    generation: str               # Respuesta generada por el LLM
    answer_grade: str             # "faithful" | "hallucination"
    sources: list[str]            # Fuentes citadas en la respuesta
    retry_count: int              # Contador de reintentos realizados
    max_retries: int              # Maximo de reintentos permitidos
    steps_trace: list[str]        # Traza de pasos ejecutados (debugging)


print("Estado RAGState definido con los siguientes campos:")
print("-" * 50)
for field_name, field_type in RAGState.__annotations__.items():
    print(f"  {field_name:20s} -> {field_type}")
print("-" * 50)
print(f"Total de campos: {len(RAGState.__annotations__)}")

---

## 6. Nodo 1: Analisis del Query

El primer nodo del pipeline analiza el query del usuario y lo refina para mejorar la calidad del retrieval. Esto es especialmente util cuando el usuario escribe queries vagos, ambiguos o con errores.

**Ejemplos de refinamiento:**
- `"precios"` → `"planes y precios de los productos NovaTech Analytics Pro, DataSync y AI Assistant"`
- `"no me funciona"` → `"problemas comunes y troubleshooting de los productos NovaTech"`
- `"cuanto cuesta el plan pro?"` → `"precio del plan Professional de Analytics Pro"`

In [ ]:
# =============================================================================
# Nodo 1: Analisis del Query
# =============================================================================

def analyze_query(state: RAGState) -> RAGState:
    """Analiza el query del usuario y genera una version refinada
    optimizada para retrieval en la base de conocimiento.
    
    - Si el query es claro y especifico, lo mantiene igual.
    - Si es vago o ambiguo, lo expande con terminos relevantes.
    - Registra el paso en la traza.
    """
    print("\n" + "=" * 60)
    print("PASO: analyze_query")
    print("=" * 60)
    
    query = state["query"]
    retry_count = state.get("retry_count", 0)
    steps_trace = state.get("steps_trace", []).copy()
    
    print(f"Query original: '{query}'")
    print(f"Intento numero: {retry_count + 1}")
    
    # Prompt para el analizador de queries
    system_prompt = """Eres un analizador de queries para un sistema RAG.
Tu base de conocimiento contiene informacion sobre productos de NovaTech Solutions:
- Analytics Pro (plataforma de business intelligence)
- DataSync (herramienta ETL)
- AI Assistant (chatbot empresarial)
- Guia tecnica (arquitectura, deploy, monitoreo, troubleshooting)

Tu tarea: dado el query del usuario, genera una version REFINADA que sea mas especifica
y optimizada para busqueda semantica en la base de conocimiento.

Reglas:
- Si el query ya es claro y especifico, devuelvelo TAL CUAL.
- Si es vago, expandelo con terminos relevantes del dominio.
- Si tiene errores de tipeo, corrigelos.
- NO inventes informacion. Solo reformula.
- Responde UNICAMENTE con el query refinado, sin explicaciones."""
    
    # Si es un re-intento, agregar contexto adicional
    user_msg = f"Query del usuario: {query}"
    if retry_count > 0:
        user_msg += f"\n\nNOTA: Este es el intento #{retry_count + 1}. "
        user_msg += "El retrieval anterior no encontro contexto relevante. "
        user_msg += "Intenta una reformulacion MAS AMPLIA o con terminos alternativos."
    
    # Llamada al LLM
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_msg),
    ])
    
    refined_query = response.content.strip()
    steps_trace.append(f"analyze_query (intento {retry_count + 1}): '{query}' -> '{refined_query}'")
    
    print(f"Query refinado: '{refined_query}'")
    print(f"Cambio aplicado: {'SI' if refined_query != query else 'NO (query ya era claro)'}")
    
    return {
        **state,
        "refined_query": refined_query,
        "steps_trace": steps_trace,
    }


print("Funcion analyze_query definida.")
print("Este nodo recibe el query y genera una version refinada para mejorar el retrieval.")

---

## 7. Nodo 2: Retrieval

El nodo de retrieval busca en ChromaDB los documentos mas relevantes para el `refined_query`. Usa busqueda semantica (similitud coseno) y retorna los top-K resultados con sus metadatos.

In [ ]:
# =============================================================================
# Nodo 2: Retrieval
# =============================================================================

def retrieve(state: RAGState) -> RAGState:
    """Busca documentos relevantes en ChromaDB usando el query refinado.
    
    Retorna los top-5 documentos mas similares junto con sus fuentes.
    """
    print("\n" + "=" * 60)
    print("PASO: retrieve")
    print("=" * 60)
    
    refined_query = state["refined_query"]
    steps_trace = state.get("steps_trace", []).copy()
    
    print(f"Buscando en ChromaDB con query: '{refined_query}'")
    
    # Buscar los top-5 documentos mas relevantes
    results = collection.query(
        query_texts=[refined_query],
        n_results=5,
        include=["documents", "metadatas", "distances"],
    )
    
    # Extraer documentos y fuentes
    retrieved_docs = results["documents"][0]   # Lista de textos
    metadatas = results["metadatas"][0]         # Lista de metadatos
    distances = results["distances"][0]         # Distancias coseno
    
    # Extraer fuentes unicas
    sources = list(set(m["source"] for m in metadatas))
    
    # Mostrar resultados del retrieval
    print(f"\nDocumentos recuperados: {len(retrieved_docs)}")
    print(f"Fuentes: {sources}")
    print(f"\nTop-3 resultados:")
    for i, (doc, meta, dist) in enumerate(zip(retrieved_docs[:3], metadatas[:3], distances[:3])):
        print(f"  [{i+1}] (distancia: {dist:.4f}) [{meta['source']}]")
        print(f"      {doc[:120]}...")
    
    steps_trace.append(f"retrieve: {len(retrieved_docs)} docs encontrados de {sources}")
    
    return {
        **state,
        "retrieved_docs": retrieved_docs,
        "sources": sources,
        "steps_trace": steps_trace,
    }


print("Funcion retrieve definida.")
print("Este nodo busca los top-5 documentos en ChromaDB por similitud semantica.")

---

## 8. Nodo 3: Evaluar Contexto (Context Grading)

Este es el primer punto de control de calidad. El LLM evalua si los documentos recuperados son **realmente relevantes** para responder el query del usuario.

Usamos **structured output** con Pydantic: el LLM debe devolver un objeto JSON con exactamente dos campos:
- `grade`: `"relevant"` o `"irrelevant"`
- `reasoning`: explicacion breve de por que

Esto elimina la ambiguedad de parsear texto libre y garantiza que siempre obtenemos una clasificacion limpia.

### Por que evaluar el contexto?

Porque el retriever puede traer documentos que son **semanticamente similares** pero **no utiles** para la pregunta. Ejemplo:
- Query: "cual es la capital de Francia?"
- Retriever devuelve: chunk sobre "regiones de AWS en eu-west-1 (Irlanda)"
- Similitud coseno alta (ambos hablan de Europa), pero totalmente irrelevante.

In [ ]:
# =============================================================================
# Modelo Pydantic para structured output del Context Grading
# =============================================================================

class ContextGrade(BaseModel):
    """Evaluacion de relevancia del contexto recuperado."""
    grade: Literal["relevant", "irrelevant"] = Field(
        description="Si el contexto recuperado es relevante para responder el query."
    )
    reasoning: str = Field(
        description="Explicacion breve de por que el contexto es o no relevante."
    )


# =============================================================================
# Nodo 3: Evaluar Contexto
# =============================================================================

def grade_context(state: RAGState) -> RAGState:
    """Evalua si los documentos recuperados son relevantes para el query.
    
    Usa structured output (Pydantic) para obtener una clasificacion limpia:
    - 'relevant': el contexto contiene informacion util para responder.
    - 'irrelevant': el contexto no ayuda a responder la pregunta.
    """
    print("\n" + "=" * 60)
    print("PASO: grade_context")
    print("=" * 60)
    
    query = state["query"]
    retrieved_docs = state["retrieved_docs"]
    retry_count = state.get("retry_count", 0)
    steps_trace = state.get("steps_trace", []).copy()
    
    # Preparar contexto para evaluacion
    context_text = "\n---\n".join(retrieved_docs)
    
    # LLM con structured output
    llm_with_structure = llm.with_structured_output(ContextGrade)
    
    system_prompt = """Eres un evaluador de calidad de retrieval para un sistema RAG.

Tu tarea: determinar si el CONTEXTO recuperado es RELEVANTE para responder el QUERY del usuario.

Criterios para "relevant":
- El contexto contiene informacion directamente util para responder la pregunta.
- No tiene que ser una respuesta completa, pero debe aportar informacion relevante.

Criterios para "irrelevant":
- El contexto no tiene relacion con la pregunta.
- La informacion recuperada es de un tema completamente diferente.
- No se podria construir una respuesta util a partir de este contexto."""
    
    user_msg = f"""QUERY del usuario: {query}

CONTEXTO recuperado:
{context_text}

Evalua si este contexto es relevante para responder el query."""
    
    # Invocar LLM con structured output
    result: ContextGrade = llm_with_structure.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_msg),
    ])
    
    print(f"Query evaluado: '{query}'")
    print(f"Contexto evaluado: {len(retrieved_docs)} documentos")
    print(f"\n>>> Resultado: {result.grade.upper()}")
    print(f">>> Razonamiento: {result.reasoning}")
    
    steps_trace.append(f"grade_context: {result.grade} - {result.reasoning}")
    
    # Incrementar retry_count si el contexto es irrelevante (evita loop infinito)
    new_retry_count = retry_count + 1 if result.grade == "irrelevant" else retry_count
    
    return {
        **state,
        "context_grade": result.grade,
        "retry_count": new_retry_count,
        "steps_trace": steps_trace,
    }


print("Funcion grade_context definida.")
print("Este nodo evalua si los documentos recuperados son relevantes para el query.")
print("Usa structured output (Pydantic) para garantizar clasificacion limpia.")

---

## 9. Nodo 4: Generar Respuesta

Si el contexto es relevante, procedemos a generar la respuesta. El LLM recibe instrucciones estrictas:

1. Responder **UNICAMENTE** con informacion del contexto proporcionado.
2. **Citar fuentes** cuando sea posible.
3. Si el contexto no contiene la respuesta, decirlo explicitamente en lugar de inventar.

In [ ]:
# =============================================================================
# Nodo 4: Generar Respuesta
# =============================================================================

def generate(state: RAGState) -> RAGState:
    """Genera una respuesta basada UNICAMENTE en el contexto recuperado.
    
    El system prompt instruye al LLM a:
    - No inventar informacion fuera del contexto.
    - Citar las fuentes cuando sea posible.
    - Admitir cuando no tiene informacion suficiente.
    """
    print("\n" + "=" * 60)
    print("PASO: generate")
    print("=" * 60)
    
    query = state["query"]
    retrieved_docs = state["retrieved_docs"]
    sources = state.get("sources", [])
    steps_trace = state.get("steps_trace", []).copy()
    
    # Construir contexto
    context_text = "\n---\n".join(retrieved_docs)
    
    system_prompt = """Eres un asistente de soporte de NovaTech Solutions.
Responde la pregunta del usuario basandote UNICAMENTE en el contexto proporcionado.

Reglas estrictas:
1. SOLO usa informacion que aparezca explicitamente en el contexto.
2. Si el contexto no contiene la respuesta, di: "No tengo informacion suficiente en la base de conocimiento para responder esta pregunta."
3. NO inventes datos, precios, fechas o politicas que no esten en el contexto.
4. Cita las fuentes cuando sea relevante.
5. Se conciso pero completo.
6. Responde en espanol."""
    
    user_msg = f"""CONTEXTO (informacion verificada de la base de conocimiento):
{context_text}

FUENTES: {', '.join(sources)}

PREGUNTA DEL USUARIO: {query}"""
    
    # Generar respuesta
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_msg),
    ])
    
    generation = response.content.strip()
    
    print(f"Query: '{query}'")
    print(f"Contexto usado: {len(retrieved_docs)} documentos")
    print(f"\n>>> Respuesta generada:")
    print(f"{generation}")
    
    steps_trace.append(f"generate: respuesta de {len(generation)} caracteres")
    
    return {
        **state,
        "generation": generation,
        "steps_trace": steps_trace,
    }


print("Funcion generate definida.")
print("Este nodo genera la respuesta usando SOLO el contexto recuperado.")

---

## 10. Nodo 5: Evaluar Respuesta (Answer Grading)

Este es el segundo punto de control de calidad. Evaluamos si la respuesta generada es **fiel al contexto** (faithfulness) o si el LLM introdujo informacion que no estaba en los documentos recuperados (hallucination).

### Por que evaluar la respuesta?

Incluso con un buen contexto, el LLM puede:
- Agregar datos de su entrenamiento que no estan en el contexto ("segun mi conocimiento...")
- Exagerar o generalizar lo que dice el contexto
- Confundir datos de diferentes chunks

El grading de respuesta detecta estos casos y permite re-generar.

In [ ]:
# =============================================================================
# Modelo Pydantic para structured output del Answer Grading
# =============================================================================

class AnswerGrade(BaseModel):
    """Evaluacion de faithfulness de la respuesta generada."""
    grade: Literal["faithful", "hallucination"] = Field(
        description="Si la respuesta es fiel al contexto o contiene alucinaciones."
    )
    reasoning: str = Field(
        description="Explicacion breve de por que la respuesta es fiel o contiene alucinaciones."
    )


# =============================================================================
# Nodo 5: Evaluar Respuesta
# =============================================================================

def grade_answer(state: RAGState) -> RAGState:
    """Evalua si la respuesta generada es fiel al contexto (no alucina).
    
    Compara la respuesta contra los documentos recuperados:
    - 'faithful': toda la informacion en la respuesta proviene del contexto.
    - 'hallucination': la respuesta contiene informacion no presente en el contexto.
    """
    print("\n" + "=" * 60)
    print("PASO: grade_answer")
    print("=" * 60)
    
    generation = state["generation"]
    retrieved_docs = state["retrieved_docs"]
    steps_trace = state.get("steps_trace", []).copy()
    retry_count = state.get("retry_count", 0)
    
    # Preparar contexto para evaluacion
    context_text = "\n---\n".join(retrieved_docs)
    
    # LLM con structured output
    llm_with_structure = llm.with_structured_output(AnswerGrade)
    
    system_prompt = """Eres un evaluador de calidad de respuestas para un sistema RAG.

Tu tarea: determinar si la RESPUESTA generada es FIEL al CONTEXTO proporcionado.

Criterios para "faithful":
- TODA la informacion factual en la respuesta se puede verificar en el contexto.
- La respuesta no agrega datos, numeros, o afirmaciones que no esten en el contexto.
- Es aceptable que la respuesta resuma o parafrasee el contexto.
- Si la respuesta dice "no tengo informacion suficiente", eso es faithful.

Criterios para "hallucination":
- La respuesta contiene informacion especifica (datos, numeros, politicas) que NO aparece en el contexto.
- La respuesta inventa o exagera mas alla de lo que dice el contexto.
- La respuesta contradice al contexto."""
    
    user_msg = f"""CONTEXTO (fuente de verdad):
{context_text}

RESPUESTA generada:
{generation}

Evalua si la respuesta es fiel al contexto."""
    
    # Invocar LLM con structured output
    result: AnswerGrade = llm_with_structure.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_msg),
    ])
    
    print(f"Respuesta evaluada ({len(generation)} chars)")
    print(f"Contexto de referencia: {len(retrieved_docs)} documentos")
    print(f"\n>>> Resultado: {result.grade.upper()}")
    print(f">>> Razonamiento: {result.reasoning}")
    
    # Incrementar retry_count si es hallucination
    new_retry_count = retry_count + 1 if result.grade == "hallucination" else retry_count
    
    steps_trace.append(f"grade_answer: {result.grade} - {result.reasoning}")
    
    return {
        **state,
        "answer_grade": result.grade,
        "retry_count": new_retry_count,
        "steps_trace": steps_trace,
    }


print("Funcion grade_answer definida.")
print("Este nodo evalua si la respuesta es fiel al contexto (faithfulness check).")

---

## 11. Funciones de Routing (Edges Condicionales)

Las funciones de routing son el "cerebro" del flujo condicional. Son funciones puras que leen el estado y devuelven un string indicando el proximo nodo.

Hay dos puntos de decision:

1. **Despues de evaluar contexto**: si es irrelevante y quedan reintentos → re-query; si no → generar.
2. **Despues de evaluar respuesta**: si es alucinacion y quedan reintentos → re-generar; si no → terminar.

In [ ]:
# =============================================================================
# Funciones de Routing (edges condicionales)
# =============================================================================

def should_regenerate(state: RAGState) -> str:
    """Decide si re-hacer el query (contexto irrelevante) o generar respuesta.
    
    Returns:
        're_query':  si el contexto es irrelevante Y hay reintentos disponibles.
        'generate':  si el contexto es relevante O se agotaron los reintentos.
    """
    context_grade = state["context_grade"]
    retry_count = state.get("retry_count", 0)
    max_retries = state.get("max_retries", 3)
    
    if context_grade == "irrelevant" and retry_count < max_retries:
        print(f"\n>>> ROUTING: Contexto irrelevante. Re-query (intento {retry_count + 1}/{max_retries})")
        return "re_query"
    else:
        if context_grade == "irrelevant":
            print(f"\n>>> ROUTING: Contexto irrelevante pero se agotaron reintentos ({retry_count}/{max_retries}). Generando con lo que hay.")
        else:
            print(f"\n>>> ROUTING: Contexto relevante. Procediendo a generar respuesta.")
        return "generate"


def should_rewrite(state: RAGState) -> str:
    """Decide si re-generar la respuesta (alucinacion) o terminar.
    
    Returns:
        're_generate':  si la respuesta alucina Y hay reintentos disponibles.
        'finish':       si la respuesta es fiel O se agotaron los reintentos.
    """
    answer_grade = state["answer_grade"]
    retry_count = state.get("retry_count", 0)
    max_retries = state.get("max_retries", 3)
    
    if answer_grade == "hallucination" and retry_count < max_retries:
        print(f"\n>>> ROUTING: Alucinacion detectada. Re-generando (intento {retry_count}/{max_retries})")
        return "re_generate"
    else:
        if answer_grade == "hallucination":
            print(f"\n>>> ROUTING: Alucinacion detectada pero se agotaron reintentos ({retry_count}/{max_retries}). Finalizando.")
        else:
            print(f"\n>>> ROUTING: Respuesta fiel al contexto. Finalizando.")
        return "finish"


print("Funciones de routing definidas:")
print("  - should_regenerate: decide entre re_query y generate")
print("  - should_rewrite: decide entre re_generate y finish")

---

## 12. Construir el Grafo

Ahora conectamos todos los nodos y edges en un `StateGraph` de LangGraph. Este es el momento donde la arquitectura cobra vida.

```
analyze_query ──→ retrieve ──→ grade_context ──┬──→ generate ──→ grade_answer ──┬──→ END
      ^                                        │                                │
      └──────── (re_query) ────────────────────┘        (re_generate) ──────────┘
```

In [ ]:
# =============================================================================
# Construccion del grafo LangGraph
# =============================================================================

# 1. Crear el grafo con el tipo de estado
workflow = StateGraph(RAGState)

# 2. Agregar nodos (cada nodo es una funcion que transforma el estado)
workflow.add_node("analyze_query", analyze_query)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_context", grade_context)
workflow.add_node("generate", generate)
workflow.add_node("grade_answer", grade_answer)

# 3. Definir el punto de entrada
workflow.set_entry_point("analyze_query")

# 4. Agregar edges fijos (transiciones siempre iguales)
workflow.add_edge("analyze_query", "retrieve")
workflow.add_edge("retrieve", "grade_context")
workflow.add_edge("generate", "grade_answer")

# 5. Agregar edges condicionales (routing basado en el estado)
workflow.add_conditional_edges(
    "grade_context",          # Nodo de origen
    should_regenerate,         # Funcion de routing
    {                          # Mapa: resultado_routing -> nodo_destino
        "re_query": "analyze_query",   # Si contexto irrelevante → volver a analizar
        "generate": "generate",         # Si contexto relevante → generar
    }
)

workflow.add_conditional_edges(
    "grade_answer",            # Nodo de origen
    should_rewrite,            # Funcion de routing
    {                          # Mapa: resultado_routing -> nodo_destino
        "re_generate": "generate",  # Si alucinacion → re-generar
        "finish": END,              # Si fiel → terminar
    }
)

# 6. Compilar el grafo
app = workflow.compile()

print("Grafo compilado exitosamente.")
print(f"\nNodos: {list(app.get_graph().nodes.keys())}")
print(f"Punto de entrada: analyze_query")
print(f"Punto de salida: END (despues de grade_answer)")
print(f"\nEdges condicionales:")
print(f"  grade_context → re_query | generate")
print(f"  grade_answer  → re_generate | finish (END)")

---

## 13. Visualizar el Grafo

LangGraph puede generar una representacion visual del grafo como diagrama Mermaid. Esto es util para documentacion y para verificar que la arquitectura esta correcta.

In [ ]:
# =============================================================================
# Visualizacion del grafo
# =============================================================================

try:
    # Intentar generar imagen PNG del grafo
    graph_image = app.get_graph().draw_mermaid_png()
    display(Image(graph_image))
    print("Grafo renderizado como imagen PNG.")
except Exception as e:
    print(f"No se pudo renderizar PNG (necesita graphviz): {e}")
    print("\nMostrando representacion Mermaid en texto:")
    print("=" * 60)
    # Fallback: mostrar mermaid como texto
    mermaid_repr = app.get_graph().draw_mermaid()
    print(mermaid_repr)
    print("=" * 60)
    print("\nPuedes copiar este texto y pegarlo en https://mermaid.live/ para ver el diagrama.")

---

## 14. Funcion Helper: Ejecutar y Mostrar Resultados

Antes de los ejemplos, creamos una funcion helper que ejecuta el grafo y muestra los resultados de forma estructurada.

In [ ]:
# =============================================================================
# Funcion helper para ejecutar y mostrar resultados
# =============================================================================

def run_self_rag(query: str, max_retries: int = 3) -> dict:
    """Ejecuta el pipeline Self-RAG completo y muestra resultados formateados.
    
    Args:
        query: Pregunta del usuario.
        max_retries: Maximo de reintentos para loops de correccion.
    
    Returns:
        Estado final del grafo.
    """
    print("\n" + "#" * 70)
    print(f"# SELF-RAG PIPELINE")
    print(f"# Query: '{query}'")
    print(f"# Max retries: {max_retries}")
    print("#" * 70)
    
    # Estado inicial
    initial_state: RAGState = {
        "query": query,
        "refined_query": "",
        "retrieved_docs": [],
        "context_grade": "",
        "generation": "",
        "answer_grade": "",
        "sources": [],
        "retry_count": 0,
        "max_retries": max_retries,
        "steps_trace": [],
    }
    
    # Ejecutar el grafo
    final_state = app.invoke(initial_state)
    
    # Mostrar resumen
    print("\n" + "=" * 70)
    print("RESUMEN FINAL")
    print("=" * 70)
    print(f"Query original:   {final_state['query']}")
    print(f"Query refinado:   {final_state['refined_query']}")
    print(f"Contexto grade:   {final_state['context_grade']}")
    print(f"Answer grade:     {final_state['answer_grade']}")
    print(f"Reintentos:       {final_state['retry_count']}/{final_state['max_retries']}")
    print(f"Fuentes:          {final_state['sources']}")
    
    print(f"\n--- TRAZA DE PASOS ---")
    for i, step in enumerate(final_state["steps_trace"], 1):
        print(f"  [{i}] {step}")
    
    print(f"\n--- RESPUESTA FINAL ---")
    print(final_state["generation"])
    print("=" * 70)
    
    return final_state


print("Funcion run_self_rag definida. Lista para ejecutar ejemplos.")

---

## 15. Ejemplo 1: Query Directo (Sin Loops)

Empezamos con un caso simple: una pregunta clara y especifica sobre los precios de un producto. Esperamos que:

1. El query NO necesite refinamiento significativo.
2. El retrieval traiga documentos relevantes.
3. El contexto sea evaluado como "relevant".
4. La respuesta sea "faithful".
5. **No haya loops** — el flujo va de principio a fin sin re-intentos.

In [ ]:
# =============================================================================
# Ejemplo 1: Query directo — flujo lineal sin loops
# =============================================================================

resultado_1 = run_self_rag(
    query="Cuanto cuesta el plan Professional de Analytics Pro?",
    max_retries=3
)

In [ ]:
# =============================================================================
# Analisis del Ejemplo 1
# =============================================================================

print("ANALISIS DEL EJEMPLO 1")
print("=" * 50)

total_pasos = len(resultado_1["steps_trace"])
tuvo_loops = resultado_1["retry_count"] > 0

print(f"Total de pasos ejecutados: {total_pasos}")
print(f"Loops de correccion: {'SI' if tuvo_loops else 'NO'}")
print(f"Reintentos usados: {resultado_1['retry_count']}")
print(f"\nObservacion:")

if not tuvo_loops:
    print("  El query era claro y especifico. El retrieval trajo documentos relevantes")
    print("  y la respuesta fue fiel al contexto. Flujo lineal optimo.")
    print("  Esto representa el 'caso feliz' del pipeline.")
else:
    print("  El pipeline necesito correccion, lo cual es inesperado para un query tan claro.")
    print("  Esto podria indicar un problema con el chunking o el threshold de relevancia.")

---

## 16. Ejemplo 2: Query Vago que Requiere Refinamiento

Ahora probamos con un query intencionalmente vago: `"precios"`. Este query es demasiado generico y podria traer documentos de cualquier producto.

Esperamos ver:
1. El analizador de query lo **refina** a algo mas especifico.
2. Posiblemente el contexto sea evaluado como irrelevante en el primer intento.
3. El sistema podria hacer un **loop de re-query** con una reformulacion mas amplia.

In [ ]:
# =============================================================================
# Ejemplo 2: Query vago — posible loop de re-query
# =============================================================================

resultado_2 = run_self_rag(
    query="precios",
    max_retries=3
)

In [ ]:
# =============================================================================
# Analisis del Ejemplo 2
# =============================================================================

print("ANALISIS DEL EJEMPLO 2")
print("=" * 50)

total_pasos = len(resultado_2["steps_trace"])
tuvo_loops = resultado_2["retry_count"] > 0

print(f"Total de pasos ejecutados: {total_pasos}")
print(f"Loops de correccion: {'SI' if tuvo_loops else 'NO'}")
print(f"Reintentos usados: {resultado_2['retry_count']}")
print(f"\nQuery original:  '{resultado_2['query']}'")
print(f"Query refinado:  '{resultado_2['refined_query']}'")
print(f"\nObservacion:")
print(f"  Nota como el analizador de queries transformo 'precios' en un query")
print(f"  mucho mas especifico y optimizado para retrieval semantico.")
print(f"  Esto demuestra el valor del primer nodo del pipeline.")

---

## 17. Ejemplo 3: Query Fuera de Dominio

Probamos con una pregunta que **no tiene respuesta** en la base de conocimiento: `"cual es la capital de Francia?"`.

Esperamos ver:
1. El retrieval trae documentos, pero son irrelevantes.
2. El grading de contexto marca como "irrelevant".
3. El sistema intenta re-query varias veces.
4. Despues de agotar reintentos, genera una respuesta honesta: "No tengo informacion suficiente."

Este es un caso critico: un RAG naive responderia con informacion inventada. Nuestro Self-RAG detecta el problema.

In [ ]:
# =============================================================================
# Ejemplo 3: Query fuera de dominio — deteccion de contexto irrelevante
# =============================================================================

resultado_3 = run_self_rag(
    query="Cual es la capital de Francia?",
    max_retries=2  # Menos reintentos para no gastar tokens innecesariamente
)

In [ ]:
# =============================================================================
# Analisis del Ejemplo 3
# =============================================================================

print("ANALISIS DEL EJEMPLO 3")
print("=" * 50)

total_pasos = len(resultado_3["steps_trace"])
tuvo_loops = resultado_3["retry_count"] > 0

print(f"Total de pasos ejecutados: {total_pasos}")
print(f"Loops de correccion: {'SI' if tuvo_loops else 'NO'}")
print(f"Reintentos usados: {resultado_3['retry_count']}/{resultado_3['max_retries']}")
print(f"Context grade:    {resultado_3['context_grade']}")
print(f"Answer grade:     {resultado_3['answer_grade']}")
print(f"\nObservacion:")
print(f"  La pregunta sobre la capital de Francia NO tiene respuesta en la base")
print(f"  de conocimiento de NovaTech. El sistema detecto esto correctamente:")
print(f"  - El context grading marco el contexto como irrelevante.")
print(f"  - El sistema intento re-queries pero la base no tiene esta info.")
print(f"  - La respuesta final es honesta sobre la falta de informacion.")
print(f"\n  CLAVE: Un RAG naive habria respondido 'Paris' usando conocimiento")
print(f"  del modelo, no de la base de datos. Eso es una ALUCINACION en contexto RAG.")

---

## 18. Comparacion: RAG Naive vs Prompt Chaining

Para entender el valor real del prompt chaining, ejecutamos el mismo query por ambos enfoques y comparamos:

| Aspecto | RAG Naive | RAG con Prompt Chaining |
|---------|-----------|------------------------|
| Pasos | 3 (query → retrieve → generate) | 5+ (con validacion y loops) |
| Validacion de contexto | Ninguna | LLM evalua relevancia |
| Validacion de respuesta | Ninguna | LLM evalua faithfulness |
| Autocorreccion | No | Si (re-query, re-generate) |
| Trazabilidad | Minima | Completa (steps_trace) |

In [ ]:
# =============================================================================
# Comparacion: RAG Naive vs Self-RAG
# =============================================================================

query_comparacion = "Que integraciones tiene AI Assistant y cuanto cuesta?"

print("=" * 70)
print("COMPARACION: RAG Naive vs Self-RAG con Prompt Chaining")
print(f"Query: '{query_comparacion}'")
print("=" * 70)

# --- RAG Naive (sin validacion) ---
print("\n" + "-" * 35)
print("ENFOQUE 1: RAG Naive")
print("-" * 35)

# Paso 1: Retrieval directo (sin refinamiento de query)
naive_results = collection.query(
    query_texts=[query_comparacion],
    n_results=5,
)
naive_context = "\n---\n".join(naive_results["documents"][0])
naive_sources = list(set(m["source"] for m in naive_results["metadatas"][0]))

# Paso 2: Generacion directa (sin validacion)
naive_response = llm.invoke([
    SystemMessage(content=f"Responde basandote en este contexto:\n{naive_context}"),
    HumanMessage(content=query_comparacion),
])

print(f"Pasos ejecutados: 2 (retrieve + generate)")
print(f"Validacion de contexto: NINGUNA")
print(f"Validacion de respuesta: NINGUNA")
print(f"Fuentes: {naive_sources}")
print(f"\nRespuesta:")
print(naive_response.content)

# --- Self-RAG (con validacion) ---
print("\n" + "-" * 35)
print("ENFOQUE 2: Self-RAG (Prompt Chaining)")
print("-" * 35)

resultado_chaining = run_self_rag(query_comparacion, max_retries=2)

# --- Resumen comparativo ---
print("\n" + "=" * 70)
print("RESUMEN COMPARATIVO")
print("=" * 70)
print(f"{'Metrica':<30} {'RAG Naive':<20} {'Self-RAG':<20}")
print("-" * 70)
print(f"{'Pasos ejecutados':<30} {'2':<20} {len(resultado_chaining['steps_trace']):<20}")
print(f"{'Validacion contexto':<30} {'No':<20} {resultado_chaining['context_grade']:<20}")
print(f"{'Validacion respuesta':<30} {'No':<20} {resultado_chaining['answer_grade']:<20}")
print(f"{'Loops de correccion':<30} {'0':<20} {resultado_chaining['retry_count']:<20}")
print(f"{'Trazabilidad':<30} {'Ninguna':<20} {'Completa':<20}")
print("=" * 70)

---

## 19. Analisis de Costos

Prompt chaining mejora la calidad, pero a un costo. Veamos cuanto cuesta realmente cada enfoque.

### Modelo de costos (gpt-4o-mini, precios aproximados)

| Recurso | Costo |
|---------|-------|
| Input tokens | ~$0.15 / 1M tokens |
| Output tokens | ~$0.60 / 1M tokens |
| Embedding (text-embedding-3-small) | ~$0.02 / 1M tokens |

In [ ]:
# =============================================================================
# Analisis de costos: RAG Naive vs Self-RAG
# =============================================================================

# Estimaciones de tokens por paso (aproximadas para gpt-4o-mini)
# Basadas en los prompts que usamos en este notebook

costos_por_millon = {
    "input": 0.15,   # USD por 1M tokens de input
    "output": 0.60,  # USD por 1M tokens de output
    "embedding": 0.02,  # USD por 1M tokens de embedding
}

# Tokens estimados por paso
tokens_por_paso = {
    "analyze_query": {"input": 350, "output": 50},
    "retrieve (embedding)": {"embedding": 30},
    "grade_context": {"input": 800, "output": 80},
    "generate": {"input": 900, "output": 200},
    "grade_answer": {"input": 1100, "output": 80},
}

print("ANALISIS DE COSTOS POR QUERY")
print("=" * 70)
print(f"Modelo: gpt-4o-mini")
print(f"Precios: input=${costos_por_millon['input']}/1M, output=${costos_por_millon['output']}/1M")
print()

# --- Costo RAG Naive ---
naive_pasos = ["retrieve (embedding)", "generate"]
costo_naive_total = 0.0

print("RAG NAIVE (2 pasos):")
print("-" * 50)
for paso in naive_pasos:
    tokens = tokens_por_paso[paso]
    costo_paso = 0.0
    for tipo, cantidad in tokens.items():
        costo_paso += (cantidad / 1_000_000) * costos_por_millon[tipo]
    costo_naive_total += costo_paso
    print(f"  {paso:<25} tokens: {tokens}  costo: ${costo_paso:.6f}")
print(f"  {'TOTAL':<25} ${costo_naive_total:.6f} por query")

# --- Costo Self-RAG (sin loops) ---
chaining_pasos = ["analyze_query", "retrieve (embedding)", "grade_context", "generate", "grade_answer"]
costo_chaining_total = 0.0

print(f"\nSELF-RAG SIN LOOPS (5 pasos):")
print("-" * 50)
for paso in chaining_pasos:
    tokens = tokens_por_paso[paso]
    costo_paso = 0.0
    for tipo, cantidad in tokens.items():
        costo_paso += (cantidad / 1_000_000) * costos_por_millon[tipo]
    costo_chaining_total += costo_paso
    print(f"  {paso:<25} tokens: {tokens}  costo: ${costo_paso:.6f}")
print(f"  {'TOTAL':<25} ${costo_chaining_total:.6f} por query")

# --- Costo Self-RAG (con 1 loop de re-query) ---
loop_pasos = ["analyze_query", "retrieve (embedding)", "grade_context",
              "analyze_query", "retrieve (embedding)", "grade_context",  # re-query
              "generate", "grade_answer"]
costo_loop_total = 0.0

print(f"\nSELF-RAG CON 1 LOOP (8 pasos):")
print("-" * 50)
for paso in loop_pasos:
    tokens = tokens_por_paso[paso]
    costo_paso = 0.0
    for tipo, cantidad in tokens.items():
        costo_paso += (cantidad / 1_000_000) * costos_por_millon[tipo]
    costo_loop_total += costo_paso
print(f"  {'TOTAL (8 pasos)':<25} ${costo_loop_total:.6f} por query")

# --- Resumen ---
print(f"\n{'=' * 70}")
print(f"RESUMEN DE COSTOS")
print(f"{'=' * 70}")
print(f"RAG Naive:              ${costo_naive_total:.6f} / query")
print(f"Self-RAG (sin loops):   ${costo_chaining_total:.6f} / query  ({costo_chaining_total/costo_naive_total:.1f}x vs naive)")
print(f"Self-RAG (1 loop):      ${costo_loop_total:.6f} / query  ({costo_loop_total/costo_naive_total:.1f}x vs naive)")
print(f"\nA 1,000 queries/dia:")
print(f"  RAG Naive:            ${costo_naive_total * 1000 * 30:.2f} / mes")
print(f"  Self-RAG (sin loops): ${costo_chaining_total * 1000 * 30:.2f} / mes")
print(f"  Self-RAG (1 loop):    ${costo_loop_total * 1000 * 30:.2f} / mes")
print(f"\nConclusion:")
print(f"  El costo adicional del Self-RAG es de centavos por query.")
print(f"  Cuando el costo de una respuesta INCORRECTA es alto (soporte al cliente,")
print(f"  chatbot medico, asesor legal), ese centavo extra esta ABSOLUTAMENTE justificado.")
print(f"  Cuando el costo de error es bajo y el volumen es masivo, RAG naive puede ser suficiente.")

---

## 20. Errores Comunes

Basandome en experiencia real implementando Self-RAG en produccion, estos son los errores mas frecuentes:

### Error 1: No poner limite de reintentos

Sin `max_retries`, un query fuera de dominio genera un **loop infinito**: el contexto siempre es irrelevante, el sistema siempre re-intenta, y nunca termina. Siempre define un tope (2-3 reintentos es razonable).

```python
# MAL: sin limite
if context_grade == "irrelevant":
    return "re_query"  # Loop infinito si la info no existe en la KB

# BIEN: con limite
if context_grade == "irrelevant" and retry_count < max_retries:
    return "re_query"
```

### Error 2: Evaluar contexto con el mismo modelo que genera

Idealmente, el evaluador deberia ser un modelo diferente (o al menos con un prompt muy distinto) al generador. Si usas el mismo modelo y prompt, tiende a evaluar sus propias respuestas como "correctas" — es como pedirle a un estudiante que se califique a si mismo.

En produccion, considera usar un modelo mas grande/critico para grading (ej. gpt-4o) y uno mas rapido/barato para generacion (ej. gpt-4o-mini).

### Error 3: Ignorar la traza de pasos

El `steps_trace` no es solo para debugging. En produccion, es tu **audit trail**. Cuando un usuario reporta una respuesta incorrecta, la traza te dice exactamente:
- Que documentos se recuperaron
- Como se evaluo el contexto
- Cuantos reintentos hubo
- Por que el sistema decidio que la respuesta era correcta

Guardala en logs estructurados.

### Error 4: Structured output sin Pydantic

Parsear la respuesta del LLM como texto libre para extraer "relevant" o "irrelevant" es fragil. El modelo puede responder "The context is somewhat relevant" y tu regex falla. Pydantic con `with_structured_output()` **garantiza** el formato.

```python
# MAL: parseo de texto libre
response = llm.invoke("Is this relevant? Answer 'relevant' or 'irrelevant'")
grade = response.content.strip().lower()  # Puede fallar con formatos inesperados

# BIEN: structured output con Pydantic
llm_structured = llm.with_structured_output(ContextGrade)
result = llm_structured.invoke(messages)  # Siempre devuelve ContextGrade valido
```

### Error 5: Chunks demasiado grandes en el context grading

Si pasas 5 chunks de 2000 tokens cada uno al evaluador de contexto, estas consumiendo muchos tokens en evaluacion. Considera evaluar cada chunk individualmente y filtrar solo los relevantes antes de pasarlos al generador. Esto es mas costoso en llamadas pero mas preciso.

---

## 21. Checklist de Comprension

Antes de pasar al Notebook 04 (RAG con Routing), verifica que puedes responder:

- [ ] **1. Cual es la diferencia fundamental entre RAG naive y Self-RAG?**
  - Pista: piensa en que pasa cuando el retrieval falla o el LLM alucina.

- [ ] **2. Por que usamos `TypedDict` para el estado y no un diccionario comun?**
  - Pista: piensa en type safety, documentacion, y tooling del IDE.

- [ ] **3. Que pasaria si eliminamos el `max_retries` del estado?**
  - Pista: piensa en el query "capital de Francia" y que pasaria sin limite.

- [ ] **4. Cuando NO vale la pena usar prompt chaining en un sistema RAG?**
  - Pista: piensa en latencia, costo, y el costo real de una respuesta incorrecta.

- [ ] **5. Como modificarias este grafo para agregar un nodo de "reformulacion de respuesta" que simplifique respuestas muy tecnicas para usuarios no-tecnicos?**
  - Pista: seria un nodo nuevo entre `generate` y `grade_answer`, o despues de `grade_answer`?

---

### Siguiente paso

En el **Notebook 04** implementaremos RAG con **Routing Multi-Dominio**: un clasificador que dirige cada query al retriever especializado (productos vs. tecnico vs. fallback). Esto complementa el prompt chaining: routing selecciona el dominio, chaining asegura la calidad dentro del dominio.

---

*Notebook creado para el curso de AI Engineering. Basado en Asai et al. (2023) Self-RAG y Chip Huyen (2024) AI Engineering.*